In [2]:
import pandas as pd

# Load the uploaded Excel file
file_path = "C:/Users/william/Desktop/薪水.xlsx"
excel_data = pd.ExcelFile(file_path)

# Display sheet names to understand the structure
sheet_names = excel_data.sheet_names
sheet_names

['工作表1']

In [3]:
# Load the sheet named '工作表1'
df = excel_data.parse('工作表1')

# Display the first few rows to understand the structure
df.head()

,degree,classification,salary,nor_salary,Unnamed: 4,Unnamed: 5,basic monthly salary,28590
0,master,1,50393,21803,1,NaN,NaN,28590
1,master,1,43984,15394,1,NaN,NaN,28590
2,master,1,44518,15928,1,NaN,NaN,28590
3,master,1,48888,20298,1,NaN,NaN,28590
4,master,1,46982,18392,1,NaN,NaN,28590


In [5]:
from scipy.stats import pearsonr

# Clean the DataFrame by removing unnecessary columns
df_clean = df[['degree', 'classification', 'salary']].dropna()

# Convert 'degree' to numerical using label encoding for Pearson correlation
df_clean['degree_num'] = df_clean['degree'].astype('category').cat.codes

# Compute Pearson correlation
corr_degree_salary, p_degree_salary = pearsonr(df_clean['degree_num'], df_clean['salary'])
(corr_degree_salary, p_degree_salary)

(0.8247915402895478, 1.872755976839596e-11)

In [7]:
from scipy.stats import f_oneway, kruskal
# 篩選 degree 為 master 的資料並取得 classification 與 salary 欄位
df_master_salary = df[df['degree'] == 'master'][['classification', 'salary']].dropna()

# 依據 classification 分組薪資
groups_salary = [group['salary'].values for _, group in df_master_salary.groupby('classification')]

# 執行 ANOVA
anova_salary_result = f_oneway(*groups_salary)

# 計算 η² 與 r
ss_total_salary = df_master_salary['salary'].var(ddof=0) * (len(df_master_salary) - 1)
ss_between_salary = sum(
    len(group) * (group['salary'].mean() - df_master_salary['salary'].mean())**2
    for _, group in df_master_salary.groupby('classification')
)
eta_squared_salary = ss_between_salary / ss_total_salary
r_effect_salary = eta_squared_salary**0.5

anova_salary_result.statistic, anova_salary_result.pvalue, eta_squared_salary, r_effect_salary

(1.9935143450807473,
 0.16517855325129413,
 0.190402267794504,
 0.4363510831824576)